In [1]:
import sys
sys.path.append('/gpfs/data/yuanlab/projects/VS_Proj20/env_xg2085/lib/python3.6/site-packages')

In [2]:
import torch
import torchvision
# import torchio as tio
import os
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [3]:
import nibabel as nib
from nibabel.testing import data_path
import nibabel.processing
import shutil
from pathlib import Path
import pandas as pd

/gpfs/share/apps/python/gpu/3.6.5/lib/python3.6/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [9]:
root=Path('..')
processed_dir=root/'data'/'processed'
ground_truth_tumor_vol = pd.read_excel('../Capstone-VS/VS_censor_send.xlsx', sheet_name='Include')
ground_truth_tumor_vol = ground_truth_tumor_vol.set_index('ID')
ground_truth_tumor_vol['target_name'] = ground_truth_tumor_vol['target_name'].apply(lambda x: x[2:].replace(' ', '-'))

In [10]:
#turn data into binary
#downsamplling to 
#256 * 256 * 208
os.mkdir('./data_processed')
dim = [256,256,208]
for file in Path(processed_dir).iterdir():
    ID = str(file).split('/')[-1]
    ID_num = ID[2:]
    os.mkdir('./data_processed/'+ID)
    for file2 in sorted(os.listdir(Path(file))):
        try:
            
            if 'image.nii.gz' in file2:
                src = '../data/processed/'+ID+'/image.nii.gz'
                dst = './data_processed/'+ID+'/image_'+str(ID_num)+'.nii.gz'
                img = nib.load(src)
                
                if list(img.header.get_data_shape()) == dim:
                    
                    print(f'Image {ID} had dimension {img.header.get_data_shape()} and spacing {img.header.get_zooms()}')
                    shutil.copyfile(src, dst)
                    print(f'No dimension change on Image {ID}')
                    print('-------------------------------------------------------------------------------------------------')

                
                else:
                    
                    size = np.multiply(list(img.header.get_data_shape()), list(img.header.get_zooms()))
                    voxel_size = np.divide(size, dim)
                    
                    resampled_img = nibabel.processing.conform(img, dim, voxel_size)
                    
                    
                    data = resampled_img.get_data()
                    header = resampled_img.header.copy()
                    
                    new_img = nib.nifti1.Nifti1Image(data.astype(int), None, header=header)
                    
                    print(f'Image {ID} had dimension {img.header.get_data_shape()} and spacing {img.header.get_zooms()}')
                    print(f'Changed Image {ID} to {new_img.header.get_data_shape()} and spacing {new_img.header.get_zooms()}')
                    print('-------------------------------------------------------------------------------------------------')
                    nib.save(new_img, dst)     
            
            
            if ground_truth_tumor_vol.loc[ID].target_name in file2:
                src = '../data/processed/'+ID+'/'+file2
                dst = './data_processed/'+ID+'/mask_VS_'+str(ID_num)+'.nii.gz'
                img = nib.load(src)
                
                if list(img.header.get_data_shape()) != dim:
                    size = np.multiply(list(img.header.get_data_shape()), list(img.header.get_zooms()))
                    voxel_size = np.divide(size, dim)
                    
                    resampled_img = nib.processing.conform(img, dim,voxel_size)
                    
                    
                    data = resampled_img.get_data() / 255
                    header = resampled_img.header.copy()
                    
                    new_img = nib.nifti1.Nifti1Image(data.astype(int), None, header=header)
                    print(f'Mask {ID} had dimension {img.header.get_data_shape()} and spacing {img.header.get_zooms()}')
                    print(f'Changed Mask {ID} to {new_img.header.get_data_shape()} and spacing {new_img.header.get_zooms()}')
                    print('-------------------------------------------------------------------------------------------------')

                    nib.save(new_img, dst)
                
                else:
                    
                    data = img.get_data() / 255
                    header = img.header.copy()
                    
                    new_img = nib.nifti1.Nifti1Image(data.astype(int), None, header=header)
                    
                    print(f'Image {ID} had dimension {img.header.get_data_shape()} and spacing {img.header.get_zooms()}')
                    print(f'No dimension change on Image {ID}')
                    print('-------------------------------------------------------------------------------------------------')
                    nib.save(new_img, dst)     
                
        except KeyError as e:
            continue  

Image VS277 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS277
-------------------------------------------------------------------------------------------------
Image VS277 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS277
-------------------------------------------------------------------------------------------------


/gpfs/share/apps/python/gpu/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:69: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


Image VS344 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS344
-------------------------------------------------------------------------------------------------
Image VS99 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS99
-------------------------------------------------------------------------------------------------
Image VS99 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS99
-------------------------------------------------------------------------------------------------
Image VS315 had dimension (256, 256, 208) and spacing (1.0, 1.0, 1.0)
No dimension change on Image VS315
-------------------------------------------------------------------------------------------------
Image VS315 had dimension (256, 256, 208) and spacing (1.0, 1.0, 1.0)
No dimension change on Image VS315
---------------------------------------------------

Image VS100 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS100
-------------------------------------------------------------------------------------------------
Image VS100 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS100
-------------------------------------------------------------------------------------------------
Image VS118 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS118
-------------------------------------------------------------------------------------------------
Image VS118 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS118
-------------------------------------------------------------------------------------------------


/gpfs/share/apps/python/gpu/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


...

Image VS183 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS183
-------------------------------------------------------------------------------------------------
Image VS183 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS183
-------------------------------------------------------------------------------------------------
Image VS11 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS11
-------------------------------------------------------------------------------------------------
Image VS11 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS11
-------------------------------------------------------------------------------------------------
Image VS188 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS188
---------------------------

Image VS331 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS331
-------------------------------------------------------------------------------------------------
Image VS329 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS329
-------------------------------------------------------------------------------------------------
Image VS329 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS329
-------------------------------------------------------------------------------------------------
Image VS74 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS74
-------------------------------------------------------------------------------------------------
Image VS74 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS74
---------------------------

Image VS75 had dimension (256, 256, 208) and spacing (1.0, 1.0, 1.0)
No dimension change on Image VS75
-------------------------------------------------------------------------------------------------
Image VS75 had dimension (256, 256, 208) and spacing (1.0, 1.0, 1.0)
No dimension change on Image VS75
-------------------------------------------------------------------------------------------------
Image VS35 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS35
-------------------------------------------------------------------------------------------------
Image VS35 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS35
-------------------------------------------------------------------------------------------------
Image VS82 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS82
---------------------------------------------------------

Image VS161 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS161
-------------------------------------------------------------------------------------------------
Image VS161 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS161
-------------------------------------------------------------------------------------------------
Image VS290 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS290
-------------------------------------------------------------------------------------------------
Image VS98 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS98
-------------------------------------------------------------------------------------------------
Image VS98 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS98
---------------------------

Image VS239 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS239
-------------------------------------------------------------------------------------------------
Image VS239 had dimension (256, 256, 208) and spacing (0.8203125, 0.8203125, 1.0)
No dimension change on Image VS239
-------------------------------------------------------------------------------------------------


In [4]:
nibabel.__version__

'3.2.1'

In [10]:
print(sys.executable)

/gpfs/share/apps/python/gpu/3.6.5/bin/python3


In [6]:
!pip install nibabel

ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/gpfs/share/apps/python/gpu/3.6.5/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2700, in _dep_map
    return self.__dep_map
  File "/gpfs/share/apps/python/gpu/3.6.5/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _Distribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/gpfs/share/apps/python/gpu/3.6.5/lib/python3.6/site-packages/pip/_internal/commands/install.py", line 517, in _warn_about_conflicts
    package_set, _dep_info = check_install_conflicts(to_install)
  File "/gpfs/share/apps/python/gpu/3.6.5/lib/python3.6/site-packages/pip/_internal/operations/check.py", line 114, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
  File "/gpfs/share/apps/python/gpu/3.6.5/lib/python

$$Dice = \frac{2\sum_{1}^{N} \hat{y_i}y_i}{\sum_{1}^{N} \hat{y_i}^2 + \sum_{1}^{N} y_i^2}$$

$$Volume = voxel_size \times \frac{Num of pixels for tumor mask}{Total count of pixels}$$